# Introduction
Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous, and can be commonly seen in online stores, movies databases and job finders. 

Within this notebook, we exemplify an implementation of systems based on **collaborative filtering**, which is also known as **User-User Filtering**. 

The notebook has 2 `Parts`: 
1. data cleaning: This part has 2 **Step**s, cleaning the `movies` dataframe and the `ratings` dataframe and a little analysis of the technique, showing it's pros and cons
2. implementing collaborative filtering recommender systems: This part is divided to 2 **Step**s too! First the user input in processed and next, learning the input's preferences is done

# Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
from math import sqrt

# Loading the data files 

In [2]:
dfMovies = pd.read_csv("movies.csv")
dfRatings = pd.read_csv("ratings.csv")
dfMovies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Part 1: Cleaning the dataframes  

## Step 1: Cleaning the `movies_df` dataframe

The first thing that comes to mind is to separate the **year attribute** in the `title` column and add it as a new column to `movies_df`

One can do so with the use of pandas and regular expressions

In [3]:
# extracting the year from the 'title' column and adding it to a new 'year' column
dfMovies['year'] = dfMovies.title.str.extract('(\(\d\d\d\d\))',expand=False)

# removing the parentheses from the 'year' column
dfMovies['year'] = dfMovies.year.str.extract('(\d\d\d\d)',expand=False)

# removing the years from the 'title' column
dfMovies['title'] = dfMovies.title.str.replace('(\(\d\d\d\d\))', '')

# applying the strip function to get rid of any ending whitespace characters that may have appeared
dfMovies['title'] = dfMovies['title'].apply(lambda x: x.strip())

/var/folders/y5/jrzb2g_51cz008jpk52hczp40000gn/T/ipykernel_98273/3959276800.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  dfMovies['title'] = dfMovies.title.str.replace('(\(\d\d\d\d\))', '')


Looking at the dataframe after the proccess above:

In [4]:
dfMovies

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017
9739,193585,Flint,Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018


With that, let's also drop the genres column since we won't need it for this type recommendation systems.

In [5]:
dfMovies = dfMovies.drop('genres', axis = 1)
dfMovies.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


## Step 2: Cleaning the `ratings_df` dataframe
Now it's time to clean the `ratings_df` dataframe.


In [6]:
dfRatings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. 

We won't be needing the `timestamp` column, so let's drop it to save memory.


In [7]:
dfRatings = dfRatings.drop('timestamp', axis = 1)
dfRatings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


# Part 2: Recommendation systems based on collaborative filtering

As hinted by the name **User-User Filtering**, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. 

There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the **Pearson Correlation Function**.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

The process for creating a User Based recommendation system is as follows:

- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbors
- Get the watched movie record of the user for each neighbor.
- Calculate a similarity score using some formula
- Recommend the items with the highest score

Let's begin by an arbitrary input user as an example to recommend movies to:

## Step1: User Input Processing

In [8]:
userInput = [
   {'title': 'Breakfast Club, The', 'rating':5},
    {'title':'Toy Story', 'rating':3.5},
    {'title':'Jumanji', 'rating':2},
    {'title':"Pulp Fiction", 'rating':5},
    {'title':'Akira', 'rating':4.5}
]

dfinputMovies = pd.DataFrame(userInput)
dfinputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


## Add `movieId` to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. 
Also, it's better to drop unnecessary columns of the input to save memory space 

In [9]:
# Filtering out the movies by title
inputId = dfMovies[dfMovies['title'].isin(dfinputMovies['title'].tolist())]

# Then merging it so we can get the movieId. It's implicitly merging it by title.
dfinputMovies = pd.merge(inputId, dfinputMovies)

# Dropping information we won't use from the input dataframe
dfinputMovies = dfinputMovies.drop('year', axis = 1)

# Final input dataframe
# If a movie you added in above isn't here, then it might not be in the original
# dataframe or it might spelled differently, please check capitalisation.
dfinputMovies.head()

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


## Step2: The users who has seen the same movies
Now with the movie ID's as input, we can now get the subset of users that have watched and reviewed the movies in it.


In [10]:
# filtering out users that have watched movies that the input has watched and storing it
userSubset = dfRatings[dfRatings['movieId'].isin(dfinputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0


We now group up the rows by user ID.

In [11]:
userSubsetGroup = userSubset.groupby(['userId'])

Let's look at one of the users, for example one with `userId` = 4.

In [12]:
userSubsetGroup.get_group(17)

,userId,movieId,rating
1667,17,1,4.5
1676,17,296,5.0


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. 

This provides a richer recommendation since we won't go through every single user.


In [13]:
# sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup, key = lambda x:len(x[1]), reverse = True)

/var/folders/y5/jrzb2g_51cz008jpk52hczp40000gn/T/ipykernel_98273/2582765367.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup, key = lambda x:len(x[1]), reverse = True)


Now let's look at the first user.


In [14]:
userSubsetGroup[0:3]

[(91,
         userId  movieId  rating
  14121      91        1     4.0
  14122      91        2     3.0
  14173      91      296     4.5
  14316      91     1274     5.0
  14383      91     1968     3.0),
 (177,
         userId  movieId  rating
  24900     177        1     5.0
  24901     177        2     3.5
  24930     177      296     5.0
  25069     177     1274     2.0
  25129     177     1968     3.5),
 (219,
         userId  movieId  rating
  31524     219        1     3.5
  31525     219        2     2.5
  31554     219      296     4.0
  31628     219     1274     2.5
  31680     219     1968     3.0)]

## Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.\
we're going to find out how similar each user is to the input through the **Pearson Correlation Coefficient**. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 \* Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.


In [15]:
userSubsetGroup = userSubsetGroup[0:150]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient.


In [16]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by = 'movieId')
    dfinputMovies = dfinputMovies.sort_values(by = 'movieId')

    #Get the N for the formula
    nRatings = len(group)

    #Get the review scores for the movies that they both have in common
    dfTemp = dfinputMovies[dfinputMovies['movieId'].isin(group['movieId'].tolist())]

    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = dfTemp['rating'].tolist()

    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()

    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [17]:
pearsonCorrelationDict

{91: 0.43852900965351443,
 177: 0.0,
 219: 0.45124262819713973,
 274: 0.716114874039432,
 298: 0.9592712306918567,
 414: 0.9376144618769914,
 474: 0.11720180773462392,
 477: 0.4385290096535153,
 480: 0.7844645405527362,
 483: 0.08006407690254357,
 599: 0.7666866491579839,
 608: 0.920736884379251,
 50: 0.15713484026367722,
 57: -0.7385489458759964,
 68: 0.0,
 103: 0.5222329678670935,
 135: 0.8703882797784892,
 182: 0.9428090415820635,
 202: 0.5222329678670935,
 217: 0.30151134457776363,
 226: 0.9438798074485389,
 288: 0.6005325641789633,
 307: 0.9655810287305759,
 318: 0.44486512077567225,
 322: 0.5057805388588731,
 330: 0.9035942578600878,
 357: 0.5606119105813882,
 434: 0.9864036607532465,
 448: 0.30151134457776363,
 469: 0.8164965809277261,
 561: 0.5222329678670935,
 600: 0.18442777839082938,
 606: 0.9146591207600472,
 610: -0.47140452079103173,
 18: 1.0,
 19: -0.5,
 21: 0,
 45: 0.5000000000000009,
 63: -0.4999999999999982,
 64: 0.0,
 66: 0.5000000000000009,
 107: -1.0,
 122: 0.86602

In [18]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient = 'index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.438529,91
1,0.000000,177
2,0.451243,219
3,0.716115,274
4,0.959271,298


## The top x similar users to input user

Now let's get the top 50 users that are most similar to the input.


In [19]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
43,1.0,132
108,1.0,33
63,1.0,305
104,1.0,17
125,1.0,117


Now, let's start recommending movies to the input user.

## Rating of selected users to all movies

We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our **pearsonDF** from the ratings dataframe and then store their correlation in a new column called \_similarityIndex". This is achieved below by merging of these two tables.


In [20]:
topUsersRating=topUsers.merge(dfRatings, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,132,1,2.0
1,1.0,132,17,3.0
2,1.0,132,29,2.0
3,1.0,132,32,3.0
4,1.0,132,34,1.5


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:


In [21]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,132,1,2.0,2.0
1,1.0,132,17,3.0,3.0
2,1.0,132,29,2.0,2.0
3,1.0,132,32,3.0,3.0
4,1.0,132,34,1.5,1.5


In [22]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,39.081916,135.336081
2,27.108493,81.524922
3,8.632175,28.868134
4,0.866025,1.732051
5,5.764409,16.394843


In [23]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.462882,1
2,3.007357,2
3,3.344248,3
4,2.000000,4
5,2.844150,5


Now let's sort it and see the top 20 movies that the algorithm recommended!


In [24]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
7579,5.0,7579
85,5.0,85
7767,5.0,7767
931,5.0,931
147328,5.0,147328
147326,5.0,147326
147300,5.0,147300
147286,5.0,147286
141816,5.0,141816


In [25]:
dfMovies.loc[dfMovies['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
76,85,Angels and Insects,1995
712,931,Spellbound,1945
4969,7579,Pride and Prejudice,1940
5013,7767,"Best of Youth, The (La meglio gioventù)",2003
9046,141816,12 Chairs,1976
9140,147286,The Adventures of Sherlock Holmes and Doctor W...,1983
9141,147300,Adventures Of Sherlock Holmes And Dr. Watson: ...,1986
9142,147326,The Adventures of Sherlock Holmes and Doctor W...,1980
9143,147328,The Adventures of Sherlock Holmes and Dr. Wats...,1979
9144,147330,Sherlock Holmes and Dr. Watson: Acquaintance,1979


## Advantages and Disadvantages of Collaborative Filtering

### Advantages

- Takes other user's ratings into consideration: Collaborative filtering leverages the collective wisdom of a large user base, providing recommendations based on the preferences of similar users. This helps in capturing the diversity of choices and preferences within a user community.
- Doesn't need to study or extract information from the recommended item: Unlike content-based filtering, collaborative filtering doesn't require detailed metadata or features of the items being recommended, making it suitable for a wide variety of products and content.
- Adapts to the user's interests which might change over time: By continuously learning from a user's interactions and feedback, collaborative filtering can dynamically adapt to evolving preferences, ensuring that recommendations remain relevant as a user's tastes and interests shift.

### Disadvantages

- Approximation function can be slow: The computational complexity of collaborative filtering algorithms can result in slow processing, especially when dealing with large datasets or intricate similarity calculations, potentially impacting real-time recommendation generation.
- There might be a low amount of users to approximate: In certain scenarios, particularly for niche or specialized products, the available user data may be limited, leading to challenges in accurately predicting preferences and generating meaningful recommendations.
- Privacy issues when trying to learn the user's preferences: Collaborative filtering heavily relies on user data, leading to potential privacy concerns and the need for robust data protection measures to ensure users' information is ethically and securely utilized for recommendation purposes.
